# Libraries

In [1]:
import cv2
# import io
import numpy as np
# import matplotlib.pyplot as plt
# import os
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile

# Constants

In [2]:
FILENAME = r'C:/Users/leohe/Documents/ITC/HIVE/Challenge/data/asl_alphabet_train.zip'

LABEL = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
         'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
         'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, 
         'del':26, 'nothing':27, 'space':28}

N = 87000 # total number of images in the training set
IMG_SIZE = 200 # original image size, to be resized
IMG_CHANNEL = 3 # rgb color scheme

# Functions

In [3]:
def idos_filter(img, *options):
    return img

- loading the images into a np.array from inside the zip file
- loading and translating the labels

In [4]:
count = 0
stacked  = np.zeros((N, IMG_SIZE, IMG_SIZE, IMG_CHANNEL), dtype='uint8')
label = np.zeros((N, 1), dtype='uint8')
with ZipFile(FILENAME) as archive:
    for entry in tqdm(archive.infolist()):
        name = str(entry).split("'")[1].replace('/', ' ').split()
        if len(name) == 3:
            with archive.open(entry) as file:
                img = idos_filter(np.array(Image.open(file)))
                stacked[count] = img
                label[count] = LABEL[name[1]]
                count += 1
                print("Image {:15s}, from label {:2d}, was read, "
                      "filtered, formatted and stacked. {:5d}/87000"
                      .format(name[-1], LABEL[name[1]], count))
            if count >= 3:
                break

  0%|                                                                                                                                        | 0/87030 [00:00<?, ?it/s]

Image space130.jpg   , from label 28, was read, filtered, formatted and stacked.     1/87000
Image space13.jpg    , from label 28, was read, filtered, formatted and stacked.     2/87000
Image space1298.jpg  , from label 28, was read, filtered, formatted and stacked.     3/87000
